In [1]:
import numpy as np

In [2]:
class Gridworld:
    def __init__(self,size=(5,5), init=(0,0), Epsilon=0, Gamma = 1, Alpha = 0.2):
      self.map = np.array([[Gridworld.Obstacle("floor") for x in range(size[0])] for y in range(size[1])])
      self.acter = self.Acteur(self, init, epsilon=Epsilon, gamma=Gamma, alpha=Alpha)
      self.wall_penalty = -0
    "returns map with objects as strings"
    def get_map(self):
      tmp = np.array([[obstacle.type for obstacle in self.map[y,:]] for y in range(self.map.shape[1])])
      return tmp

    "returns map shape"
    def get_map_shape(self):
      return self.map.shape
    
    "places an object(Obstacle) at a position"
    def set_object(self, object, position):
      self.map[position] = object

    "parent class for all Obstacles"
    class Obstacle:
      "init fixes type, reward and code(for a color map)"
      def __init__(self, type, code=0, reward=None):
        self.type = type
        if reward is not None:
          self.reward = reward
        else:
          self.reward = 0
        self.code = code
      "What happens when i land on the Field, Will be overwritten"
      def action(self):
        return None



    class pit(Obstacle):
      "A pit"
      def __init__(self, reward = -50):
        code = 1
        super().__init__("pit", code,  reward)
      def action(self):
        return "End"

    class exit(Obstacle):
      "An Exit"
      def __init__(self, reward = 100):
        code = 100
        super().__init__("exit", code, reward)
      def action(self):
        return "End"

    class center(Obstacle):
      "An 'Center' Object"
      def __init__(self, reward = 5):
        code = 10
        super().__init__("center", code, reward)

    class corner(Obstacle):
      "for 9.1 an corner object"
      def __init__(self, reward = 1):
          code = 15
          super().__init__("corner", code, reward)

    class Acteur:
      def __init__(self,parent, init, epsilon=0, gamma=1, alpha=0.2):
        self.gridworld = parent
        self.pos = init
        self.init_pos = init
        self.map_shape = parent.get_map_shape()
        self.moves = {0:"up", 1:"right", 2:"down", 3:"left"}
        self.Q = self.make_Q_Matrix()
        self.epsilon = epsilon
        self.gamma = gamma
        self.alpha = alpha

      def get_map(self):
        tmp = self.gridworld.get_map()
        tmp[self.pos] = "XXXXX"
        return tmp
      "Moving when possible"
      def step(self,direction):
        if direction == "up":
          if self.pos[0] > 0:
            self.pos = (self.pos[0]-1, self.pos[1])
          else:
            raise Exception("Invalid Move")
        if direction == "down":
          if self.pos[0] < self.map_shape[0]-1:
            self.pos = (self.pos[0]+1, self.pos[1])
          else:
            raise Exception("Invalid Move")
        if direction == "left":
          if self.pos[1] > 0:
            self.pos = (self.pos[0], self.pos[1]-1)
          else:
            raise Exception("Invalid Move")
        if direction == "right":
          if self.pos[1] < self.map_shape[1]-1:
            self.pos = (self.pos[0], self.pos[1]+1)
          else:
            raise Exception("Invalid Move")

      "Attempts to move in a direction, returns the Reward for that step. Does not interact with the tile"
      def move(self, direction):
          try:
            self.step(direction)
            return self.gridworld.map[self.pos].reward
            "if the move is not possible"
          except:
            return self.gridworld.wall_penalty
      
      def make_Q_Matrix(self):
        "Q_Matrix is a 3-D Array"
        Q_Matrix = np.zeros(shape=(self.map_shape[0],self.map_shape[1], len(self.moves)))
        return Q_Matrix
    
      def make_V_Matrix(self):
        pass

      def make_episode(self, algorithm=True, max_steps = 10000):
        "returns an episode and trains the Q matrix in the process."

        "return to start"
        self.pos = self.init_pos

        "define the transistion rules"

        def get_move_Q():
          epsilon = self.epsilon
          "if random number is smaller than epsilon, explore, else go for max Q"
          if np.random.rand() > epsilon:
            possible_moves = np.argwhere(self.Q[self.pos] == np.max(self.Q[self.pos]))
            move = possible_moves[np.random.randint(0,high=len(possible_moves))]
            move = move[0]
          else:
            move = np.random.randint(0, high=len(possible_moves))
          return move
        
        def get_move_uniform():
          return np.random.randint(0, len(self.moves))


        if algorithm == True:
          next_step = get_move_Q
        else:
          next_step = get_move_uniform
        trace = [(self.pos, 0)]

        curr_action = next_step()
        step = 0
        while step < max_steps:
          old_pos = self.pos
          "gets action to do"
          next_action = next_step()
          "moves us and returns reward"
          reward = self.move(self.moves[curr_action])
          "check interactions"
          if self.gridworld.map[self.pos].action() == "End":
            break;

          "saves new position and the rewardobtained"
          trace.append((self.pos,reward))

          """Update Q matrix
          for Q learning we take max_a'(q(s',a')), which is
          automatically the next step for epsilon = 0"""
          self.Q[old_pos][curr_action] = self.Q[old_pos][curr_action]*(1-self.alpha) +\
           self.alpha*(reward + self.gamma*self.Q[self.pos][next_action])

          "update steps, action"
          step += 1
          curr_action = next_action

        return trace 

In [3]:
"make a map"
grid = Gridworld()

In [4]:
"modify the map for exercise 9.1"
grid.set_object(Gridworld.corner(), (0,0))
grid.set_object(Gridworld.corner(), (0,4))
grid.set_object(Gridworld.corner(), (4,0))
grid.set_object(Gridworld.corner(), (4,4))
grid.set_object(Gridworld.center(), (2,2))
grid.get_map()

array([['corner', 'floor', 'floor', 'floor', 'corner'],
       ['floor', 'floor', 'floor', 'floor', 'floor'],
       ['floor', 'floor', 'center', 'floor', 'floor'],
       ['floor', 'floor', 'floor', 'floor', 'floor'],
       ['corner', 'floor', 'floor', 'floor', 'corner']], dtype='<U6')

a)
For the Problem stated here the Transistionmatrix is 0.25 for each direction and p(s'/s,a) = 1 for s+a=s', 0 else.
(with s = (x,y), a in {(0,1),(1,0),(0,-1),(-1,0)}

In [5]:
"b&c"
traces = []
returns = []
for episode in range(10):

  trace = (grid.acter.make_episode(algorithm=False))
  traces.append(trace)
  return_val = 0
  for i in trace:
    "in this deterministic problem we can use gamma=1"
    
    return_val += grid.acter.gamma*i[1]
  returns.append(return_val)

In [6]:
returns

[2638, 2980, 2946, 2678, 2836, 2930, 2538, 2919, 2816, 2750]

Now we calculate the State-Value-function, which is just the max Q-value for each state.

In [7]:
Q = grid.acter.Q
V = np.sum(Q, axis = 2)

In [8]:
V

array([[220.4281004 , 223.31420419, 224.67802944, 222.63219224,
        221.24053919],
       [222.72990231, 225.41878095, 228.89167963, 223.50416498,
        223.2030323 ],
       [223.61628755, 228.85394148, 226.8041611 , 230.12346783,
        225.70020747],
       [224.01341699, 226.24521118, 229.89447316, 227.66896762,
        224.23738772],
       [221.96637381, 224.11886174, 222.99767811, 224.60410107,
        222.83977574]])